In [1]:
import os            # linux shell
import re            #^[]?%
import pymatgen as mg  #materials project
import pymatgen.analysis.diffraction as anadi
import pymatgen.analysis.diffraction.xrd as xrd
import numpy as np
import glob          #random
import matplotlib.pyplot as plt
import torch         #machine learning
import torch.nn as nn
from torch.autograd import Variable

In [2]:
torch.set_default_dtype(torch.float64)

torch.set_printoptions(precision=16)

patt_xrd = xrd.XRDCalculator('CuKa')

train_path='/home/mii/trainCSSO/O4/train/'

test_path='/home/mii/trainCSSO/O4/test/'

global sample_num, rmat_num, atoms_num
sample_num=1
rmat_num=28  #row nums of the matrix for the input of CNN 
atoms_num=24

global move_num,extend_num
extend_num=100

LR_D=0.001  #learning rate
LR_G=0.001

In [3]:
move_num=-121

In [4]:
def random_xxpsk(file_path):
    folder=np.random.choice(glob.glob(file_path +"*"))
    return folder

In [5]:
def tomgStructure(folder):
    POSfile=folder+'/CONTCAR'
    R_mgS=mg.Structure.from_file(POSfile)
    return R_mgS
    #print(POSfile)
    '''
    with open(POSfile,'r+') as pp:
        transfer=pp.readlines()
        if transfer[5] =='   Ca   Sn   S    O \n':
            pass
        else:
            transfer.insert(5,'Ca   Sn   S    O\n')
            pp.seek(0)# move the zhizhen
            pp.truncate()#read by the byte
            pp.writelines(transfer)
        pp.close()
    '''   

In [6]:
def get_xrdmat4(mgStructure):
    global rmat_num
    xrd_data4 =patt_xrd.get_pattern(mgStructure)
    i_column = rmat_num
    xxx=[]
    yyy=[]
    mat4=[]
    xrd_i=len(xrd_data4)
    for i in range(xrd_i):
        if xrd_data4.y[i]>1 and xrd_data4.y[i] < 20:
            xxx.append(xrd_data4.x[i])
            yyy.append(xrd_data4.y[i])
    mat4.append(np.asarray(xxx))
    mat4.append(np.asarray(yyy))
    mat4=np.asarray(mat4)
    
    xrd_x=[]
    xrd_y=[]
    xrd_mat4=[]
    xrow=len(mat4[0])
    
    if xrow < i_column:
        for i in mat4[0]:
            xrd_x.append(i)
        for j in mat4[1]:
            xrd_y.append(j)
        for i in range(0,i_column-xrow):
            xrd_x.append(0)
            xrd_y.append(0)
        xrd_x=np.asarray(xrd_x)
        xrd_y=np.asarray(xrd_y)
    if xrow > i_column:
        xrd_x=mat4[0][:i_column]
        xrd_y=mat4[1][:i_column]
    if xrow == i_column:
        xrd_x= mat4[0]
        xrd_y= mat4[1]
        
    xrd_x=np.sin(np.dot(1/180*np.pi,xrd_x))
    xrd_y=np.dot(1/100,xrd_y)
    xrd_mat4.append(xrd_x)
    xrd_mat4.append(xrd_y)
    xrd_mat4=np.array(xrd_mat4)
    return xrd_mat4

In [7]:
def get_energy(folder):
    energy_string=os.popen('grep TOTEN '+folder+'/OUTCAR | tail -1').read().split(' ')[-2]
    energy=np.float64(float(energy_string))
    return energy

def linear_transform(energy):
    global extend_num, move_num
    energy_transform=(energy-move_num)*extend_num
    return energy_transform
def inverse_transform(energy_transform):
    global extend_num, move_num
    energy=energy_transform/extend_num+move_num
    return energy
def get_energy_per_atom(energy):
    energy_per_atom=energy/atoms_num
    return energy_per_atom

In [ ]:
get_energy(train_path+'0000')

In [8]:
def GANs_Gmat(Random_Structure):
    global rmat_num
    RS_xrdmat = get_xrdmat4(Random_Structure)
    multimat3_RS =  np.zeros((rmat_num,rmat_num),dtype='float32')
    multimat3_RS = np.asarray(np.sqrt(np.dot(RS_xrdmat.T, RS_xrdmat)))
    return multimat3_RS

In [11]:
class GcssNet(nn.Module):
    def __init__(self, input_size=(sample_num,28,28)):
        super(GcssNet, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(#(3,28,28)
                in_channels=sample_num,
                out_channels=32,
                kernel_size=5,
                stride=1,
                padding=2,
            ),#->(32,28,28)
            nn.ReLU(),#->(32,28,28)
            nn.MaxPool2d(kernel_size=2),
        )#->(#->(32,14,14))
        self.conv2=nn.Sequential(#->(32,14,14))
            nn.Conv2d(
                in_channels=32,
                out_channels=64,
                kernel_size=5,
                stride=1,
                padding=2,
            ),#->(64,14,14)
            nn.ReLU(),#->(64,14,14)
            nn.MaxPool2d(kernel_size=2),#->(64,7,7)
        )
        self.out=nn.Sequential(
            nn.Linear(64*7*7,128),
            nn.ReLU(),
            nn.Linear(128,sample_num),            
        )
        
    def forward(self,x):
        x=self.conv1(x)
        x=self.conv2(x) #batch(64,7,7)
        x=x.view(x.size(0),-1) #(batch, 64*7*7)
        output=torch.unsqueeze(self.out(x),dim=0)
        return output

class DNet(nn.Module):
    def __init__(self):
        super(DNet, self).__init__()
        self.Dlstm=nn.LSTM(
            input_size=sample_num,
            hidden_size=32,
            num_layers=1,
            batch_first=True,
        )
        self.out=nn.Sequential(
            nn.Linear(32,10),
            nn.ReLU(),
            nn.Linear(10,1),
            nn.Sigmoid(),
        )
        #nn.Linear(32,1)
        #nn.Relu
        #nn.Linear
        #nn.Sigmoid
        
    def forward(self,x):
        D_out,(h_n,h_c)=self.Dlstm(x,None)
        out = self.out(D_out[:,-1,:]) #(batch,time step,input)   
        return out

In [12]:
G1=GcssNet()
D1=DNet()

mat_Gl=[]    
mat_Dl=[]
pre_dd=[]
pre_gg=[]

In [121]:
opt_D1=torch.optim.Adam(D1.parameters(),lr=0.0001)
opt_G1=torch.optim.Adam(G1.parameters(),lr=0.0001)

In [138]:
file_path=train_path
tfset=[]
for step in range(1,81):       
 
    sample_path=[]
    for i in range(1,sample_num + 1):
        path_ = random_xxpsk(file_path)
        sample_path.append(path_)
        tfset.append(path_)
    E_Gibbs=[]
    for path1_ in sample_path:
        try:
            total_energy=get_energy(path1_)
            tt_energy=linear_transform(total_energy)
            #print(samp_Gibbs)
        except:
            print(path1_)
            
        E_Gibbs.append(tt_energy)
        
    E_Gibbs=np.asarray(E_Gibbs,dtype=np.float64)       
    E_Gibbs=Variable(torch.from_numpy(E_Gibbs[np.newaxis,np.newaxis,:]),requires_grad=True)
    #print(tfactor.shape)
    
    g_in=[]
    for path2_ in sample_path:
        path2_=str(path2_)                
        
        try:
            tomgS=tomgStructure(path2_)
            #print(tomgS)
            gin=GANs_Gmat(tomgS)
            #print(gin)
        except:
            pass
        g_in.append(gin)
       
    g_in=np.asarray(g_in)
    g_in=g_in[np.newaxis,:,:,:] 
    g_in=np.asarray(g_in,dtype=np.float64) 
    g_in=Variable(torch.from_numpy(g_in),requires_grad=True)
    
    Gout=G1(g_in)
    #print(Gout.shape)
    
    prob_Tfactor_mat0=D1(E_Gibbs)
    prob_G1_mat1=D1(Gout)
    
    D1_loss=-torch.mean(torch.log(prob_Tfactor_mat0)+torch.log(1.-prob_G1_mat1))
    G1_loss=torch.mean(torch.log(1.-prob_G1_mat1))
    
    
    if step%2==0:
        opt_D1.zero_grad()
        D1_loss.backward(retain_graph=True)
        opt_D1.step()
        
        opt_G1.zero_grad()
        G1_loss.backward()
        opt_G1.step()
    else:
        opt_D1.zero_grad()
        D1_loss.backward()
        opt_D1.step()
    


    if step%8==0:
        print(step)
        gg=G1_loss.data.numpy().mean()
        dd=D1_loss.data.numpy().mean()
        print(dd)
        print(gg)
        print(prob_Tfactor_mat0.data.numpy().mean())
        print(prob_G1_mat1.data.numpy().mean())
        mat_Gl.append(gg)
        mat_Dl.append(dd)
        pre_dd.append(prob_Tfactor_mat0.data.numpy().mean())
        pre_gg.append(prob_G1_mat1.data.numpy().mean())
    
    
    if step%40==0:
        print('----------------------------------test-------------------------')
        file_path_test=test_path
        E_Gibbs=[]
        E_Gmodel=[]
        for step_test in range(10):
            sample_path=[]
            for i in range(1,sample_num + 1):
                path_ = random_xxpsk(file_path_test)
                sample_path.append(path_)
            
            for path1_ in sample_path:
                try:
                    total_energy=get_energy(path1_)
                    tt_energy=linear_transform(total_energy)
            #print(samp_Gibbs)
                except:
                    print(path1_)
                    
            tt_energy=float(tt_energy)
            print('DFT',tt_energy)
            E_Gibbs.append(tt_energy)
        
                   

        #print(tfactor.shape)
    
            g_in=[]
            for path2_ in sample_path:
                path2_=str(path2_)                
        
                try:
                    tomgS=tomgStructure(path2_)
            #print(tomgS)
                    gin=GANs_Gmat(tomgS)
            #print(gin)
                except:
                    pass
                g_in.append(gin)
       
            g_in=np.asarray(g_in)
            g_in=g_in[np.newaxis,:,:,:] 
            g_in=np.asarray(g_in,dtype=np.float64) 
            g_in=Variable(torch.from_numpy(g_in),requires_grad=True)
    
            Gout=G1(g_in)
        #print(Gout.shape)
    
            G_data=Gout.data.numpy().mean()
            print('G_predict',G_data)
            E_Gmodel.append(G_data)

        
            
            
            print('error',abs(tt_energy-G_data))
            
        print('------------------end-test----------------------------')
        print(abs(np.asarray(E_Gibbs)-np.asarray(E_Gmodel)).mean())
    else:
        pass

8
1.401504472251232
-0.6841875728729455
0.48806001488039513
0.49550006744381175
16
1.3902138783785543
-0.6851603818182316
0.49408214185402066
0.49599061084990603
24
1.386631540508357
-0.6835513416771273
0.49505807560977594
0.49517898671847516
32
1.3868334199448749
-0.6813850603560246
0.4938870855959336
0.4940842170254604
40
1.4017003965979837
-0.6809169392978757
0.4863710556982582
0.49384733175263373
----------------------------------test-------------------------
DFT -30.42805399999935
G_predict -32.56314950777571
error 2.135095507776363
DFT 7.060803999999621
G_predict -37.2672552480705
error 44.32805924807012
DFT -25.325427999999306
G_predict -35.861775510878026
error 10.53634751087872
DFT -43.18116400000065
G_predict -35.40098149586267
error 7.780182504137976
DFT -72.86394999999999
G_predict -35.10085203890493
error 37.763097961095056
DFT -40.5848430000006
G_predict -35.632223806159374
error 4.95261919384123
DFT -34.782327000000635
G_predict -36.04182592025359
error 1.259498920252959

In [139]:
torch.save(G1.state_dict(),"/home/mii/trainCSSO/G1_o4_14.pkl") 
torch.save(D1.state_dict(),"/home/mii/trainCSSO/D1_o4_14.pkl")

In [142]:
E_Gibbs_test=[]
E_Gmodel_test=[]
abserrset=[]
err0set=[]
testfile=[]
for m1,n1,fname in os.walk(test_path):
    for ieach in n1:
        ieach=test_path+ieach
        testfile.append(ieach)
        
for path_ in testfile:
    try:
        GGG=get_energy(path_)
        GGG=get_energy_per_atom(GGG)
        E_Gibbs_test.append(GGG)
        g_in=[]
        tomgS=tomgStructure(path_)
        gin=GANs_Gmat(tomgS)
        g_in.append(gin)
        g_in=np.asarray(g_in)
        g_in=g_in[np.newaxis,:,:,:]
        g_in=np.asarray(g_in,dtype=np.float64)
        g_in=Variable(torch.from_numpy(g_in),requires_grad=True)
        Gout=G1(g_in)
        G_data=Gout.data.numpy().mean()
        G_data=inverse_transform(G_data)
        G_data=get_energy_per_atom(G_data)
        E_Gmodel_test.append(G_data)
        #print(G_data)
        #print(GGG)
        abserr=abs(G_data-GGG)
        abserrset.append(abserr)
        err0=abs(abserr/GGG)
        err0set.append(err0)
    except:
        print(path_)

In [144]:
print(np.asarray(err0set).mean())

0.0017560357104249786


In [148]:
print(np.asarray(abserrset).mean())

0.008149945170311262


In [145]:
E_Gibbs_t=[]
E_Gmodel_t=[]
abs_t_errset=[]
err_t_0set=[]
testfile=[]
for m1,n1,fname in os.walk(train_path):
    for ieach in n1:
        ieach=train_path+ieach
        testfile.append(ieach)
#'''        
for path_ in testfile:
    try:
        GGG=get_energy(path_)
        GGG=get_energy_per_atom(GGG)
        E_Gibbs_t.append(GGG)
        g_in=[]
        tomgS=tomgStructure(path_)
        gin=GANs_Gmat(tomgS)
        g_in.append(gin)
        g_in=np.asarray(g_in)
        g_in=g_in[np.newaxis,:,:,:]
        g_in=np.asarray(g_in,dtype=np.float64)
        g_in=Variable(torch.from_numpy(g_in),requires_grad=True)
        Gout=G1(g_in)
        G_data=Gout.data.numpy().mean()
        G_data=inverse_transform(G_data)
        G_data=get_energy_per_atom(G_data)
        E_Gmodel_t.append(G_data)
        #print(G_data)
        #print(GGG)
        abserr=abs(G_data-GGG)
        abs_t_errset.append(abserr)
        err0=abs(abserr/GGG)
        err_t_0set.append(err0)
    except:
        print(path_)

In [146]:
print(np.asarray(abs_t_errset).mean())

0.005674220317829688


In [147]:
print(np.asarray(err_t_0set).mean())

0.0012226919561405935


In [149]:
print(mat_Dl)

[1.4246751356763363, 1.4061022022301817, 1.4075383017967666, 1.3952667665057221, 1.3958012440087795, 1.3840881897425779, 1.383777584862563, 1.3801161218283844, 1.3735736866707897, 1.3763928990701593, 1.3613309719588842, 1.3627818659599162, 1.3587826354672, 1.3504359315371688, 1.3544415436368917, 1.3587689471309323, 1.3627875095533204, 1.340471504219476, 1.3572924043974814, 1.3660189179289972, 1.3525094816673568, 1.3676580067951207, 1.373059522778668, 1.3664621786173976, 1.385559153199985, 1.4404140418566755, 1.4054169868153965, 1.3970375216430062, 1.4372711404166587, 1.4025693451009666, 1.4130565677360547, 1.4237438666040536, 1.4514129064121024, 1.4376411604018302, 1.439935228698063, 1.432170374823769, 1.4260521856329844, 1.436307624873581, 1.40854904416729, 1.4235324679569046, 1.4340218220712417, 1.4175855168309086, 1.496228030533861, 1.4458279669678453, 1.4124243403521408, 1.4126265079684064, 1.4099274254997995, 1.4078031014265733, 1.4055017899459272, 1.428975972063095, 1.40117464238

In [151]:
print(pre_dd)

[0.53199203220911, 0.5407913156690403, 0.5389506792030893, 0.5446392769732741, 0.5433170902417048, 0.5486666073705065, 0.5482227063934757, 0.5492068112817982, 0.551801280639204, 0.5504702989348652, 0.5578840573084306, 0.5558380818595466, 0.5597446932614032, 0.5638541818672282, 0.5602643674321675, 0.5622143724776864, 0.5601387661269978, 0.5715781758656404, 0.5724987376404618, 0.5674399060691965, 0.5733382633937956, 0.5768563597094899, 0.572101570893424, 0.5770543928805146, 0.5778366568447935, 0.5483702670166672, 0.5664026118952977, 0.5826590922153388, 0.5609874802432863, 0.5783988593316716, 0.5791092078373588, 0.5718043238031795, 0.5561550317094789, 0.5652944351727393, 0.5625393749141498, 0.5651786772698847, 0.5697976792342074, 0.5625602308574995, 0.5759378199837433, 0.5674388656356981, 0.5597894874059262, 0.5672940525702225, 0.5234782362191275, 0.5482176005766878, 0.5645872684096536, 0.5625642866902517, 0.5619295916178833, 0.5607679851992153, 0.5600147346311644, 0.5446011532763285, 0.5

In [152]:
print(mat_Gl)

[-0.7935483688731413, -0.7913803895435735, -0.7894070852754265, -0.7876351859362141, -0.7857390744518025, -0.7838238952561445, -0.7827039087991667, -0.7808359188514272, -0.779006390308587, -0.7794106219067193, -0.7777268511668837, -0.7755036191386024, -0.7785081313676744, -0.7774763278365395, -0.7750950216873242, -0.7828968910004075, -0.783216780259211, -0.7811174897832516, -0.7995476558464814, -0.7993984903063419, -0.7962300827303048, -0.8174960200184442, -0.8146207909043774, -0.8166434301241402, -0.8370951023366029, -0.8396094913989692, -0.8369668615992022, -0.856884510493319, -0.8592144498513059, -0.8550777648516052, -0.8667923631137158, -0.864785429221971, -0.8647047168234241, -0.8672326010276123, -0.8646410811920037, -0.8615570199493574, -0.863578255758322, -0.8610505513176769, -0.8567934686367582, -0.8569102067754159, -0.8538273407978533, -0.850708018379154, -0.8489682073956787, -0.8447449774621111, -0.8407640271824928, -0.837376643983868, -0.8335487067442179, -0.8293550688020389

In [153]:
print(pre_gg)

[0.5477627596767557, 0.5467812551313245, 0.5458860336659958, 0.5450806761335919, 0.5442172800902275, 0.5433435380881309, 0.5428318025203376, 0.5419770188078841, 0.5411382856813678, 0.5413237346007868, 0.5405507783681229, 0.5395281798275132, 0.5409095967611477, 0.5404356611998296, 0.5393399937718522, 0.5429200193330134, 0.5430662109254362, 0.5421059666106669, 0.5504677385761458, 0.5504006788521848, 0.5489739058512776, 0.5584641315600252, 0.5571927879499445, 0.5580875220166885, 0.5670335784355184, 0.5681208569768077, 0.56697805094198, 0.5755175048570382, 0.576505372047029, 0.574749879716081, 0.5797024487206224, 0.578858092320011, 0.5788240995747139, 0.5798874389149771, 0.5787972969275587, 0.5774962768140749, 0.5783493940028502, 0.5772822377340483, 0.5754788574233625, 0.5755284123388371, 0.5742178041460992, 0.5728875785744756, 0.5721438368951347, 0.5703330810119611, 0.5686191891750535, 0.5671554593569733, 0.5654953823126034, 0.5636694011860163, 0.5620758714890524, 0.5601292701254771, 0.55

In [154]:
print(err0set)

[0.00028811348121086466, 0.00352950805117636, 0.0008088159623282823, 0.0017501516777721632, 0.0015062611587064012, 0.004582904262759234, 0.004700760136084852, 0.002054342861986378, 0.0024246106639524687, 0.0004060747734799634, 0.002415768697937243, 0.0024857925174396286, 0.0029347801054476867, 0.0020809590760089937, 0.0034787250391931095, 0.0018642993122469401, 0.002739383113506639, 0.00015981338775589095, 0.004006834822412845, 0.0014161239186124314, 0.0018066293631671228, 0.0006398070992309742, 0.0015809707843841233, 0.002913926392047397, 0.0022893636558087844, 0.002142033273655264, 0.0007673614182234902, 4.116089784476279e-05, 0.0015438468264412212, 0.004067624801449358, 0.0021217108578962768, 0.0020016824024114434, 0.0015380869911917458, 0.0023610375094303008, 0.003368843869795605, 0.003529531467104571, 0.0012922803172005042, 0.0005671983009102274, 0.00023375313822970712, 0.0034734995326786908, 0.00021018202800505346, 0.006633096994845436, 0.004750942441836106, 0.0008751472265965097

In [155]:
print(abserrset)

[0.001337753617850268, 0.016323808029201636, 0.0037563686113930572, 0.008108141523543999, 0.0069805400232496595, 0.02117237694664542, 0.021717199674735177, 0.009513101050703021, 0.011277553341090929, 0.0018854497558784544, 0.011184481581308248, 0.011564191146113956, 0.013661616011328093, 0.009676506878784075, 0.016084895510035224, 0.008668121448376809, 0.012678877083581952, 0.0007414482968677305, 0.018670461623699453, 0.006580971322207141, 0.008399911563758522, 0.0029680033327652566, 0.007324537055444225, 0.013483404014303702, 0.010648655765569615, 0.009917209185067222, 0.0035582309005910417, 0.00019099004486733406, 0.007155139256046716, 0.01895489574403708, 0.009825533241843765, 0.009270149818646267, 0.00712615634639846, 0.010984216138214542, 0.015688634559350767, 0.01632391594586835, 0.005990383440924063, 0.002633765833868651, 0.0010853003948465556, 0.016175824452134613, 0.0009756115416994504, 0.030592525288929195, 0.022150082468412435, 0.004064821351489556, 0.0014297799428995006, 0.

In [156]:
print(abs_t_errset)

[0.003490514741352868, 0.006575249260686533, 0.008849363215233552, 0.006575717282421678, 0.0076294305957675945, 0.014479152919949279, 0.0028510567233928086, 0.014480099586616113, 0.0022009826457143333, 0.010557583906255275, 0.002101783348262387, 0.003836880440035273, 0.0011627436938210778, 0.0018853568392120934, 0.0028502075567260476, 0.010159954213438738, 0.009307230287880053, 0.008745273063166081, 0.008756636664541517, 0.006574784677352952, 0.010671656265136775, 0.008756472081207889, 0.0022010326457140295, 0.007813459219647712, 0.0018503852445723723, 0.007698406002743852, 0.006457685995053808, 0.011923533459611946, 0.006869393770677412, 0.019541008636750234, 0.007113305111259294, 0.0017872044968418876, 0.004047591893752589, 0.002254017416788656, 0.002003892789502082, 0.0007422910227257162, 0.0022551574167888333, 0.01452045492890175, 0.0051160663768330394, 0.0023265345902787615, 0.010557131822920951, 0.0051120201370729035, 0.005116063876832833, 0.004210716133091985, 0.0053994525577030

In [157]:
print(err_t_0set)

[0.0007527197583784024, 0.0014188255840176286, 0.0019032727547712931, 0.001418771640050331, 0.0016465479387503161, 0.0031105493952143866, 0.0006146856136941802, 0.003110752134515317, 0.00047448334293593913, 0.0022802656605780515, 0.0004531613787635177, 0.0008275690974901448, 0.0002506006327595754, 0.00040605476987433636, 0.0006145024215179046, 0.002194161968906355, 0.002009091638668232, 0.0018876585098685067, 0.0018901795363683253, 0.0014187251927070975, 0.002294788197951451, 0.0018901439427871428, 0.00047449412694826505, 0.001686124055626401, 0.00039861188041038023, 0.0016563406619681227, 0.0013932742801478584, 0.0025756516441661844, 0.001482443600869507, 0.00419382835051566, 0.0015352766895429199, 0.0003852458174910565, 0.0008728009858043924, 0.0004859219941041256, 0.0004316003615815807, 0.00015992661360022972, 0.00048616787525058574, 0.003119394838573836, 0.0011010963256781574, 0.0005010807620051724, 0.002280167795321288, 0.0011026210406385637, 0.0011010957882124938, 0.0009064776315